# Activating the Julia Environment

In [1]:
using Pkg; Pkg.activate(".")

  Activating project at `~/code/julia/notebooks`


# The problem

|            |    Product A   | Product  B   | Capacity  |
| :----:     | :------------: | :---------:  | :------:  |
| Labour     |     7          |       9      |   1250    |
| Material 1 |     4          |       8      |   3600    |
| Material 2 |     6          |       4      |   5655    |
| Unit Profit|     60         |      50      |           |          

- Labour should be used in production.
- if Material 1 is used then Material 2 is not used.
- if Material 2 is used then Material 1 is not used.
- Formulate the problem as a mixed integer problem

$$
\begin{aligned}
z = \begin{Bmatrix}
1 & , & \text{if Material 1 is used} \\
0 & , & \text{if Material 1 is not used (otherwise)}\\
\end{Bmatrix}
\end{aligned}
$$

$$
\begin{aligned}
\max\quad & 60 x + 50 y\\
\text{Subject to} \quad & 7 x + 9 y \leq 1250\\
& 4 x + 8 y \leq 3600 + M(1-z)\\
& 6 x + 4 y \leq 5655 + Mz\\
& x,y \ge 0 \\
& z \in \{0, 1\}
\end{aligned}
$$

- if a constraint has a very big number in its right hand side and the constraint type is $\le$ then the constraint is disabled.
- if $z=1$ then $M(1-z)$ is zero, so the right hand side of the first constraint is enabled.
- if $z=0$ then $M(1-z)$ is very big number, so the right hand side of the first constraint is disabled.

## Julia Solution

In [2]:
using JuMP, HiGHS

In [8]:
begin 
    bigM = 100000
    model = Model(HiGHS.Optimizer)
    MOI.set(model, MOI.Silent(), true)
    @variable(model, x >= 0)
    @variable(model, y >= 0)
    @variable(model, z, Bin)
    @objective(model, Max, 60*x + 50*y)
    @constraint(model, 7x + 9y <= 1250)
    @constraint(model, 4x + 8y <= 3600 + bigM * (1-z))
    @constraint(model, 6x + 4y <= 5655 + bigM * z)
    optimize!(model)
    println("Feasible: ", JuMP.is_solved_and_feasible(model))
    println("Objective: ", JuMP.objective_value(model))
    println("z = ", value(z))
    println("x = ", value(x))
    println("y = ", value(y))
end

Feasible: true
Objective: 10714.285714285716
z = 0.0
x = 178.57142857142858
y = 0.0
